# **GPEN dfl colab by deepartist**
Sources: 
[bycloud - youtube channel](https://www.youtube.com/channel/UCfg9ux4m8P0YDITTPptrmLg) and 
[video]( https://www.youtube.com/watch?v=2HdFV4k_CCY)

[Tendeepfake](mailto:tendeepfake@gmail.com
)
## *There is no need for colab pro to use it.*

[Contact me deepartist](mailto:deepartist.ai@gmail.com)



---



# Install

In [ ]:
#@title Check if you have loaded GPU
!nvidia-smi

**Mount your Google drive with auth key**

1. Connect to your google drive to be able to import and export files to/from google drive
2. Click on Link below
3. Login and confirm
4. Ctrl+V auth key and hit return key

**OR mount your Google drive permantly**

1. Create new notebook
2. Change runtime to GPU
3. Click on files, and mount your google drive permantly 
4. Click in first cell
5. Select all cells with shift+down
6. Copy all cells from this notebook
7. No more auth token coyping

**OR SIMPLY SKIP** if you dont want import and export files to/from google drive

In [ ]:
#@title Connect to your gdrive OR permantly mount it

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone DFL Git and install needed dependencies
!git clone https://github.com/iperov/DeepFaceLab.git /content/DeepFaceLab
!pip install colorama
print("\nDone")

In [ ]:
#@title Install all dependencies and download weights from org repo
#@title START
%cd /content
!git clone https://github.com/bycloudai/GPEN
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth && mv RetinaFace-R50.pth GPEN/weights/
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth && mv GPEN-512.pth GPEN/weights/
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-1024-Color.pth && mv GPEN-1024-Color.pth GPEN/weights/
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
!pip install pip install opencv-python
%cd GPEN
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!touch "/content/GPEN/__init__.py"

In [ ]:
#@title create custom upscale function
%cd /content/GPEN
!source /content/GPEN/__init__.py
from face_enhancement import FaceEnhancement
import tqdm 
import glob 
import cv2
import numpy as np

def upscale(indir='examples/imgs', outdir= 'examples/outs', model={'name':'GPEN-512', 'size':512}, filetype='jpg'):
  os.makedirs(outdir, exist_ok=True)

  faceenhancer = FaceEnhancement(size=model['size'], model=model['name'], channel_multiplier=2)
  
  files = sorted(glob.glob(os.path.join(indir, '*.*g')))
  print(f"Found {len(files)} files")
  for file in tqdm.tqdm(files[:]):
      filename = os.path.basename(file)
      
      im = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
      if not isinstance(im, np.ndarray): print(filename, 'error'); continue
      #im = cv2.resize(im, (0,0), fx=1, fy=1) #no rescale

      img, orig_faces, enhanced_faces = faceenhancer.process(im)
      cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'.jpg'), img)
      

# Upscale Process

In [ ]:
#@title 0. Delete old example and result files
!rm -rf /content/GPEN/examples/imgs
!mkdir -p /content/GPEN/examples/imgs
!rm -rf /content/GPEN/examples/outs
!mkdir -p /content/GPEN/examples/outs
!rm -rf /content/GPEN/examples/dfl
print("\nDone!")

## Upload files
You can upload single or multiple files or your zip aligend folder or faceset.pak

### files and zipped aligned folder

In [ ]:
#@title 1a. Upload one or more (DFL).jpg files to upscale *OR* 
%cd /content/GPEN/examples/imgs/
from google.colab import files
uploaded = files.upload()
%cd /content/GPEN/
print("\nDone!")

In [ ]:
#@title 1b. Upload aligned.zip and extract to imgs *OR*
%cd /content/
!rm aligned.zip
from google.colab import files
uploaded = files.upload()
%cd /content/GPEN/
!7z e '/content/aligned.zip' -o'/content/GPEN/examples/imgs'
print("\nDone!")

In [ ]:
#@title 1c. copy aligned.zip from your drive and extract to imgs
import os

file_on_drive = "/content/drive/MyDrive/aligned.zip" #@param {type:"string"}
%cd /content/
!rm aligned.zip
!cp $file_on_drive /content/
!7z e '/content/aligned.zip' -o'/content/GPEN/examples/imgs'
print("\nDone!")

### faceset.pak

In [ ]:
#@title 2b. Upload faceset.pak and extract to imgs *OR*
%cd /content/GPEN/examples/imgs/
!rm faceset.pak
from google.colab import files
uploaded = files.upload()
%cd /content
cmd = "DeepFaceLab/main.py util --input-dir /content/GPEN/examples/imgs --unpack-faceset"
!python $cmd
print(f"Unpacking done!")
print("\nDone!")

In [ ]:
#@title 2d. copy faceset.pak from your drive and extract to imgs
import os

file_on_drive = "/content/drive/MyDrive/faceset.pak" #@param {type:"string"}
%cd /content/
!rm faceset.pak
!cp $file_on_drive /content/GPEN/examples/imgs/
cmd = "DeepFaceLab/main.py util --input-dir /content/GPEN/examples/imgs/ --unpack-faceset"
!python $cmd
print(f"Unpacking done!")
print("\nDone!")

## Upscaling
run both 

In [ ]:
#@title 3. Optional Resize if original faceset if too small
%cd /content/
cmd = "DeepFaceLab/main.py facesettool resize --input-dir /content/GPEN/examples/imgs"
!python $cmd

In [ ]:
#@title 4. Save DFL metadata and Start upscaling and restore
%cd /content/
cmd = "DeepFaceLab/main.py util --input-dir /content/GPEN/examples/imgs --save-faceset-metadata"
!python $cmd
%cd /content/GPEN
print("Start upscaling")
upscale()
print("Start upscaling done")
%cd /content/
!cp /content/GPEN/examples/imgs/meta.dat /content/GPEN/examples/outs
cmd = "DeepFaceLab/main.py util --input-dir /content/GPEN/examples/outs --restore-faceset-metadata"
!python $cmd
print("\nDone!")

## Save your enhanced files

### Zips

In [ ]:
#@title 5a. Zip enhanced faces
%rm /content/dfl_enhanced.zip
%rm /content/GPEN/examples/outs/faceset.pak
!7z a -tzip /content/dfl_enhanced.zip -w /content/GPEN/examples/outs/*
print("\nDone!")

In [ ]:
#@title 5b. Download dfl_enhanced.zip *OR*
from google.colab import files
files.download("/content/dfl_enhanced.zip")
print("\nDone!")

In [ ]:
#@title 5c. Copy dfl_enhanced.zip to gdrive
from google.colab import files
!cp -f /content/dfl_enhanced.zip /content/drive/MyDrive
print("\nDone!")

### Faceset.pak

In [ ]:
#@title 6a. Repack to faceset.pak 
%cd /content
import glob 
files = glob.glob("/content/GPEN/examples/outs/*jpg")
if len(files) > 0:
  %rm /content/GPEN/examples/outs/faceset.pak
  cmd = "DeepFaceLab/main.py util --input-dir /content/GPEN/examples/outs --pack-faceset"
  !python $cmd
  !cp /content/GPEN/examples/outs/faceset.pak /content/faceset.pak
else: 
  print('No imgs found')


In [ ]:
#@title 6b. Download faceset.pak *OR*
from google.colab import files
files.download("/content/GPEN/examples/outs/faceset.pak")
print("\nDone!")

In [ ]:
#@title 6c. Copy faceset.pak to gdrive
from google.colab import files
!cp -f /content/GPEN/examples/outs/faceset.pak /content/drive/MyDrive/
print("\nDone!")